In [ ]:
using BitemporalPostgres
using LifeInsuranceDataModel, LifeInsuranceProduct
using SearchLight
using SearchLightPostgreSQL
using TimeZones
ENV["SEARCHLIGHT_USERNAME"] = "mf"
ENV["SEARCHLIGHT_PASSWORD"] = "mf"
SearchLight.Configuration.load() |> SearchLight.connect

SearchLight.query("DROP SCHEMA public CASCADE")
SearchLight.query("CREATE SCHEMA public")
LifeInsuranceDataModel.load_model()

In [ ]:
using LifeInsuranceDataModel, LifeInsuranceProduct, SearchLight
import SearchLight: Serializer.serialize

tif = get_tariff_interface(Val(1))
tif.parameters


LifeRiskTariff = create_tariff("Life Risk Insurance", 1, 0.02, "", serialize(tif.parameters))
ProfitParticipationTariff = create_tariff(
    "Profit participation", 2, 0.02,
    "", "{}"
)

In [ ]:
cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end

p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(
    ref_tariff=LifeRiskTariff,
    ref_role=ppRole["Main Coverage - Life"],
    description="Main Coverage - Life",
)

pp2 = ProductPart()
ppr2 = ProductPartRevision(
    ref_tariff=ProfitParticipationTariff,
    ref_role=ppRole["Profit participation"],
    description="Profit participation Life Risk",
)

w0 = Workflow(
    type_of_entity="Product",
    tsw_validfrom=ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

In [ ]:
prs = prsection(1, now(tz"UTC"), ZonedDateTime(2023, 4, 1, 21, 0, 1, 1, tz"UTC"), 0)

In [ ]:

# create partner 1
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1",sex="f",smoker=false)
w = Workflow(type_of_entity="Partner",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)
Partner1 = p.id

In [ ]:
w1 = Workflow(
    type_of_entity="Contract",
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"UTC"),
)

create_entity!(w1)
c = Contract()
cr = ContractRevision(description="contract creation properties")
create_component!(c, cr, w1)

cpr = ContractPartnerRef(ref_super=c.id)
cprr = ContractPartnerRefRevision(
    ref_partner=Partner1,
    ref_role=cpRole["Policy Holder"],
    description="policiyholder ref properties",
)
create_subcomponent!(c, cpr, cprr, w1)
# pi 1
LifeRiskProduct = find(Product, SQLWhereExpression("id=?", 1))[1].id.value
PartnerRole = tiprRole["Insured Person"]

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(
    ref_product=1,
    description="from contract creation",
)
create_subcomponent!(c, cpi, cpir, w1)

LifeInsuranceDataModel.create_product_instance(
    w1,
    cpi,
    LifeRiskProduct,
    Partner1.value,
    PartnerRole,
)
commit_workflow!(w1)

In [ ]:
cid = 1
h = find(Contract, SQLWhereExpression("id =?", cid))[1].ref_history
vi = find(ValidityInterval, SQLWhereExpression("ref_history=?", h), order=["ValidityInterval.id"])[1];
txntime = vi.tsdb_validfrom
reftime = vi.tsworld_validfrom

cs = csection(cid, txntime, reftime)

ti = cs.product_items[1].tariff_items[1]


In [ ]:
using JSON, LifeInsuranceProduct,MortalityTables,LifeContingencies,Yields
import LifeContingencies: ä, A
tariffparms = JSON.parse(ti.tariff_ref.rev.parameters)
mts = tariffparms["mortality_tables"]
interface_id = ti.tariff_ref.ref.revision.interface_id
parms = get_tariff_interface(Val(interface_id)).calls
args=parms["calculation_target"]
args["selected"]="ä"
args["ä"]["m"]["value"] = "10"
args["ä"]["n"]["value"] = "20"
args["ä"]["frequency"]["value"] = "2"
args["ä"]["begin"]["value"] = string(Date("2023-04-01"))
parms
parms["calculation_target"]
calculate!(Val(1),ti,parms)
parms["result"]

In [ ]:
tariffparms = JSON.parse(ti.tariff_ref.rev.parameters)
mts = tariffparms["mortality_tables"]
interface_id = ti.tariff_ref.ref.revision.interface_id
tgt = get_tariff_interface(Val(interface_id)).calls["calculation_target"]
tgt["selected"] = "ä"
parms = tgt[tgt["selected"]]
parms["m"] = 10
parms["n"] = 20
parms["frequency"] = 2
parms["begin"] = string(Date("2023-04-01"))

# accessiong partner LifeInsuranceDataModel
dob = ti.partner_refs[1].ref.revision.date_of_birth
smoker = ti.partner_refs[1].ref.revision.smoker ? "smoker" : "nonsmoker"
sex = ti.partner_refs[1].ref.revision.sex
#accessing tariff data
i = ti.tariff_ref.ref.revision.interest_rate

issue_age = LifeInsuranceProduct.insurance_age(dob, Date(parms["begin"]))

life = SingleLife(
    mortality=MortalityTables.table(mts[sex][smoker]).select[issue_age])

yield = Yields.Constant(i)      # Using a flat

lc = LifeContingency(life,yield)  # LifeContingenc


r1 = ä(lc)
r2 = ä(lc,1)
r3 = ä(lc, 2)
r4 = ä(lc, 30)

println("r1")
println(r1)
println("r2")
println(r2)
println("r3")
println(r3)
println("r4")
println(r4)
r4

In [ ]:
using LifeInsuranceProduct
get_tariff_interface(Val(2))
calculate!(Val(2), ti, parms)
parms["result"]